In [1]:

import numpy as np
from allennlp_models import pretrained
from allennlp.predictors import Predictor
from allennlp.interpret.saliency_interpreters import SimpleGradient, IntegratedGradient, SmoothGradient
from allennlp.interpret.attackers import Hotflip, input_reduction
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
predictor = pretrained.load_predictor("lm-next-token-lm-gpt2")

lerc is not a registered model.


In [3]:
tokenizer = predictor._dataset_reader._tokenizer.tokenizer

In [91]:

class AutoRegressiveGPT():
    def __init__(self, predictor: Predictor):
        self.predictor = predictor
    
    def get_next_token(self, inputs):
        np.random.seed(42)
        outputs = predictor.predict(inputs)
        probabilities = np.array(outputs["probabilities"])
        probabilities = probabilities / probabilities.sum()
        top_tokens = outputs["top_tokens"]
        top_tokens = np.array(top_tokens).reshape(-1)
        token = np.random.choice(top_tokens, 1, p=probabilities)[0]
        token = token.replace("Ġ", " ")
        token = token.replace("Ċ", "\n")
        token = token.replace("Âł", "\xa0")
        
        return token
    
    def generate(self, inputs, max_length: int =10):
        for _ in tqdm(range(max_length)):
            token = self.get_next_token(inputs)
            inputs += token
                
        return inputs
    
    def generate_and_saliency_score(self, inputs, interpreters: list[Predictor], max_length: int =10):
        grad_matrix_list = [[] for i in range(len(interpreters))]
        for _ in tqdm(range(max_length)):
            token = self.get_next_token(inputs)
            for i, interpreter in enumerate(interpreters):
                interpretation = interpreter.saliency_interpret_from_json({"sentence": inputs})
                grads = np.array(interpretation['instance_1']['grad_input_1'])
                grad_matrix_list[i].append(grads)
            
            inputs += token
                
        return inputs, grad_matrix_list
    
    def get_score_from_grad_matrix(self, grad_matrix: list[list]):
        n_inputs = len(grad_matrix[0])
        n_outputs = len(grad_matrix)

        scores = np.zeros((n_inputs, n_outputs))

        for n in range(n_outputs):
            if n == 0:
                scores[:, n] = grad_matrix[n]
            else:
                sum_ = 0
                for j in range(1,n+1):
                    sum_ += scores[:, n-j]*grad_matrix[n][-j]
                scores[:, n] = grad_matrix[n][:n_inputs] + sum_
        return scores
    
    def get_scores_from_grad_matrix_list(self, grad_matrix_list: list[list]):
        scores_list = []
        for grad_matrix in tqdm(grad_matrix_list):
            scores_list.append(self.get_score_from_grad_matrix(grad_matrix))
            
        return scores_list
    
    def get_padded_grad_matrix(self, grad_matrix_list):
        padded_grad_matix_list = []
        for grad_matrix in grad_matrix_list:
            max_len = len(grad_matrix[-1])
            padded_grad_matrix = [np.pad(grad_vec, (0, max_len-len(grad_vec)), 'constant') for grad_vec in grad_matrix]
            padded_grad_matix_list.append(np.array(padded_grad_matrix))
        
        return padded_grad_matix_list
    
    def get_scores_from_text(self, text: str, interpreters: list[Predictor], max_length: int = 10):
        result, grad_matrix = self.generate_and_saliency_score(text, interpreters, max_length)
        scores_list = self.get_scores_from_grad_matrix_list(grad_matrix)
        padded_grad_matrix_list = self.get_padded_grad_matrix(grad_matrix)
        
        return result, scores_list, padded_grad_matrix_list

In [5]:
class Gpt2Predictor(Predictor):
    """
    The HuggingFace implementation of GPT-2 is not an AllenNLP model;
    however, our demo only expects an AllenNLP ``Predictor``. Accordingly,
    we implement a ``Predictor`` that wraps the HuggingFace GPT-2 implementation.
    """
    def __init__(self,
                 model_name: str = "gpt2",
                 cache_size: int = 0) -> None:
        """
        Each cache element is about 8MB, so size accordingly.
        """
        # Cache stores tuples, so default value is a tuple
        self.tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
        self._model = GPT2LMHeadModel.from_pretrained(model_name)

        # The end of text marker.
        self.END_OF_TEXT = self.tokenizer.encoder["<|endoftext|>"]


    def get_prob_of_word(self, inputs: dict, word: str) -> float:
        previous_str = inputs["previous"]
        next_str = inputs.get("next")
        topk = inputs.get("topk", 10)

        logits = self._predict(previous_str, next_str)
        probabilities = torch.nn.functional.softmax(logits, dim=0)
        
        word_id = self.tokenizer.encode(word)
        word_prob = probabilities[word_id[0]]
        
        return word_prob

    def predict_json(self, inputs: dict) -> dict:
        previous_str = inputs["previous"]
        next_str = inputs.get("next")
        topk = inputs.get("topk", 10)

        logits = self._predict(previous_str, next_str)
        probabilities = torch.nn.functional.softmax(logits, dim=0)

        best_logits, best_indices = logits.topk(topk)
        best_words = [self.tokenizer.decode([idx.item()])
                      for idx in best_indices]
        best_probabilities = probabilities[best_indices].tolist()
        
        next_str = best_words[0]

        return {
            "logits": best_logits.tolist(),
            "probabilities": best_probabilities,
            "words": best_words,
            "output": previous_str + (next_str or "")
        }

    def _predict(self, previous: str, next: str = None) -> torch.Tensor:

        past_logits, past = (None, None)

        # CASE 1: Previously seen input, no next
        if next is None and past is not None:
            return past_logits

        # CASE 2: Previously seen input, yes next
        elif past is not None:
            token_ids = self.tokenizer.encode(next)
        # CASE 3: Brand new input, no next
        elif next is None:
            token_ids = self.tokenizer.encode(previous)
        # CASE 4: Brand new input, yes next
        else:
            token_ids = self.tokenizer.encode(previous) + self.tokenizer.encode(next)

        inputs = torch.LongTensor([token_ids])

        result = self._model(inputs)
        logits = result.logits.squeeze(dim=0)
        new_token_logit = logits[-1, :]
        key = previous if next is None else previous + next

        return new_token_logit

    def __getitem__(self, index: int) -> str:
        return self.tokenizer.decode([index])

In [22]:
def generate_heat_map_from_scores(scores, tokenizer, result, h=600, w=600):
    tokens = tokenizer.encode(result)
    words = [tokenizer.decode(token) for token in tokens]
    n_input = scores.shape[0]
    n_output = scores.shape[1]
    
    scores_round = np.round(scores, 3)
    
    fig = px.imshow(
            scores_round,
            text_auto=True,
            y=words[:n_input],
            x=words[n_input:],
            color_continuous_scale='Greys',
            width=w, height=h,
            )

    fig.update_yaxes(autorange="reversed")
    fig.update_xaxes(side="top")
    fig.update_layout(
    yaxis = dict(
    tickfont = dict(size=12)))
    fig.update_layout(
    xaxis = dict(
    tickfont = dict(size=12)))
    
    return fig

def generate_heat_map_from_grad_matrix(scores, tokenizer, sample_text, result, h=600, w=600):
    tokens = tokenizer.encode(result)
    words = [tokenizer.decode(token) for token in tokens]
    tokens_input = tokenizer.encode(sample_text)
    words_input = [tokenizer.decode(token) for token in tokens_input]
    
    scores_round = np.round(scores, 3)
    
    fig = px.imshow(
            scores_round,
            text_auto=True,
            y=words[:-1],
            x=words[len(words_input):],
            color_continuous_scale='Greys',
            width=w, height=h,
            )

    fig.update_yaxes(autorange="reversed")
    fig.update_xaxes(side="top")
    fig.update_layout(
    yaxis = dict(
    tickfont = dict(size=12)))
    fig.update_layout(
    xaxis = dict(
    tickfont = dict(size=12)))
    
    return fig

def generate_heat_map_from_raking_scores(scores, sample_text: str, tokenizer, result: str):
    
    tokens = tokenizer.encode(sample_text)
    input_words = [tokenizer.decode(token) for token in tokens]
    
    fig = px.imshow(
                scores.reshape(1, -1),
                text_auto=True,
                y=[""],
                x=input_words,
                color_continuous_scale='Greys',
                )
    fig.update_xaxes(side="top")
    fig.update_xaxes(title_text=result)
    
    return fig

def generate_heat_map_from_weights(scores, sample_text: str, tokenizer, result: str):
    
    tokens = tokenizer.encode(sample_text)
    input_words = [tokenizer.decode(token) for token in tokens]

    tokens_output = tokenizer.encode(result)
    outputs_words = [tokenizer.decode(token) for token in tokens_output]
    
    outputs_words = outputs_words[len(input_words):]
    fig = px.imshow(
                scores.reshape(1, -1),
                text_auto=True,
                y=[""],
                x=outputs_words,
                color_continuous_scale='Greys',
                )
    fig.update_xaxes(side="top")
    fig.update_xaxes(title_text="Output words weights")
    
    return fig

In [27]:
def get_output_words_weigths(sample_text: str, result: str, predictor_hugging: Predictor, null_token: str, add_previous: bool):
    tokens_input = tokenizer.encode(sample_text)
    tokens_output = tokenizer.encode(result)
    words = [tokenizer.decode(token) for token in tokens_output]

    input_words = words[:len(tokens_input)]
    output_words = words[len(tokens_input):]

    add_previous = add_previous

    input_str = ""
    for _ in tokens_input:
        input_str += "|<endoftext>|"
        
    probs = []
    for word in tqdm(output_words):
        prob = predictor_hugging.get_prob_of_word({"previous": input_str}, word)
        probs.append(prob.item())
        if add_previous:
            input_str += word
            
    probs = np.array(probs)
    w = 1/probs
    w = w/w.sum()
    return w

In [8]:
predictor_hugging = Gpt2Predictor()

In [92]:
interpreter_smooth = SmoothGradient(predictor)
interpreter_simple = SimpleGradient(predictor)
interpreter_integrated = IntegratedGradient(predictor)

generator = AutoRegressiveGPT(predictor=predictor)

In [105]:
sample_text = "Question: What teams Cristiano Ronaldo played? Answer:"
result = generator.generate(sample_text, max_length=50)
print(result)

100%|██████████| 50/50 [00:07<00:00,  6.72it/s]

Question: What teams Cristiano Ronaldo played? Answer: Cristiano Ronaldo.


The answer is that he played for Real Madrid.


The question is: What teams Cristiano Ronaldo played? Answer: Cristiano Ronaldo.The answer is that he played for Real Madrid.The question is:


In [103]:
result

"Cristiano Ronaldo is a great player. He's a great player. He's a great player. He's a great player. He's a great player. He's a great player. He's a great player. He's a great player. He's a great player."

In [94]:
predictor.predict("def main():\n\n\nfrom django.contrib.auth import auth_token, auth_token_key, auth_token_secretĊĊĊĊ")

{'token_ids': [4299,
  1388,
  33529,
  628,
  198,
  6738,
  42625,
  14208,
  13,
  3642,
  822,
  13,
  18439,
  1330,
  6284,
  62,
  30001,
  11,
  6284,
  62,
  30001,
  62,
  2539,
  11,
  6284,
  62,
  30001,
  62,
  21078,
  128,
  232,
  128,
  232,
  128,
  232,
  128,
  232],
 'top_indices': [[128], [628], [198], [11], [13]],
 'probabilities': [0.40771692991256714,
  0.09306562691926956,
  0.07746997475624084,
  0.03918102756142616,
  0.02111506089568138],
 'top_tokens': [['Ä'], ['ĊĊ'], ['Ċ'], [','], ['.']],
 'tokens': ['def',
  'Ġmain',
  '():',
  'ĊĊ',
  'Ċ',
  'from',
  'Ġdj',
  'ango',
  '.',
  'cont',
  'rib',
  '.',
  'auth',
  'Ġimport',
  'Ġauth',
  '_',
  'token',
  ',',
  'Ġauth',
  '_',
  'token',
  '_',
  'key',
  ',',
  'Ġauth',
  '_',
  'token',
  '_',
  'secret',
  'Ä',
  'Ĭ',
  'Ä',
  'Ĭ',
  'Ä',
  'Ĭ',
  'Ä',
  'Ĭ']}

In [95]:
predictor_hugging.predict_json({"previous": "def main():\n\n\nfrom django.contrib.auth import auth_token, auth_token_key, auth_token_secretĊĊĊĊ"})

{'logits': [-40.7717399597168,
  -42.24896240234375,
  -42.43244171142578,
  -43.1141357421875,
  -43.732330322265625,
  -43.81690979003906,
  -43.858070373535156,
  -43.96296691894531,
  -44.41304016113281,
  -44.643798828125],
 'probabilities': [0.40771597623825073,
  0.09306967258453369,
  0.07746832072734833,
  0.03918033838272095,
  0.021114930510520935,
  0.019402481615543365,
  0.018620077520608902,
  0.01676584593951702,
  0.010689591988921165,
  0.008486798964440823],
 'words': ['�', '\n\n', '\n', ',', '.', '_', ' from', ' �', '(', ';'],
 'output': 'def main():\n\n\nfrom django.contrib.auth import auth_token, auth_token_key, auth_token_secretĊĊĊĊ�'}

In [16]:
result, scores_list, grad_matrix_list = generator.get_scores_from_text(
    sample_text, 
    interpreters=[interpreter_simple, interpreter_integrated, interpreter_smooth],
    max_length=9
    )

  0%|          | 0/9 [00:00<?, ?it/s]/usr/local/lib/python3.10/site-packages/torch/nn/modules/module.py:1053: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
100%|██████████| 3/3 [00:00<00:00, 980.05it/s]


In [19]:
fig = generate_heat_map_from_grad_matrix(grad_matrix_list[2].T, tokenizer, sample_text, result, h=700)

fig.update_layout(title_text="Smooth Gradient")
fig.show()

In [20]:
fig = generate_heat_map_from_scores(scores_list[2], tokenizer, result)

fig.update_layout(title_text="Smooth Gradient")
fig.show()

In [24]:
w = get_output_words_weigths(sample_text, result, predictor_hugging, "|<endoftext>|", True)
ranking_score = scores_list[2]@w 
fig = generate_heat_map_from_raking_scores(ranking_score, sample_text, tokenizer, result)
fig.show()

100%|██████████| 9/9 [00:01<00:00,  4.93it/s]


In [25]:
fig = generate_heat_map_from_weights(w, sample_text, tokenizer, result)
fig.show()

In [28]:
w = get_output_words_weigths(sample_text, result, predictor_hugging, "|<endoftext>|", False)
ranking_score = scores_list[2]@w 
fig = generate_heat_map_from_raking_scores(ranking_score, sample_text, tokenizer, result)
fig.show()

100%|██████████| 9/9 [00:01<00:00,  6.35it/s]


In [29]:
fig = generate_heat_map_from_weights(w, sample_text, tokenizer, result)
fig.show()

In [45]:
sample_text = "Climate change has critical consequences such as"
result = generator.generate(sample_text, max_length=4)
print(result)

100%|██████████| 4/4 [00:00<00:00, 16.34it/s]

Climate change has critical consequences such as the loss of biodiversity


In [32]:
result, scores_list, grad_matrix_list = generator.get_scores_from_text(
    sample_text, 
    interpreters=[interpreter_simple, interpreter_integrated, interpreter_smooth],
    max_length=4
    )

  0%|          | 0/4 [00:00<?, ?it/s]/usr/local/lib/python3.10/site-packages/torch/nn/modules/module.py:1053: UserWarning:

Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.

100%|██████████| 3/3 [00:00<00:00, 3103.06it/s]


In [33]:
fig = generate_heat_map_from_grad_matrix(grad_matrix_list[2].T, tokenizer, sample_text, result, h=700)

fig.update_layout(title_text="Smooth Gradient")
fig.show()

In [34]:
fig = generate_heat_map_from_scores(scores_list[2], tokenizer, result)

fig.update_layout(title_text="Smooth Gradient")
fig.show()

In [35]:
w = get_output_words_weigths(sample_text, result, predictor_hugging, "|<endoftext>|", True)
ranking_score = scores_list[2]@w 
fig = generate_heat_map_from_raking_scores(ranking_score, sample_text, tokenizer, result)
fig.show()

100%|██████████| 4/4 [00:01<00:00,  3.23it/s]


In [36]:
fig = generate_heat_map_from_weights(w, sample_text, tokenizer, result)
fig.show()

In [42]:
w = get_output_words_weigths(sample_text, result, predictor_hugging, "|<endoftext>|", False)
ranking_score = scores_list[2]@w 
fig = generate_heat_map_from_raking_scores(ranking_score, sample_text, tokenizer, result)
fig.show()

100%|██████████| 4/4 [00:00<00:00,  6.60it/s]


In [43]:
fig = generate_heat_map_from_weights(w, sample_text, tokenizer, result)
fig.show()